In [ ]:
from smach_utils2 import *

In [ ]:
tfBuffer = tf2.Buffer()

listener = tf2.TransformListener(tfBuffer)
b_tf=tf2.TransformBroadcaster()
b_st=tf2.StaticTransformBroadcaster()


In [ ]:
def write_tf(pose, q, child_frame , parent_frame='map',time=0):
    t= TransformStamped()
    t.header.stamp = rospy.Time.now()
    t.header.frame_id =parent_frame
    t.child_frame_id =  child_frame
    t.transform.translation.x = pose[0]
    t.transform.translation.y = pose[1]
    t.transform.translation.z = pose[2]
    #q = tf.transformations.quaternion_from_euler(eu[0], eu[1], eu[2])
    t.transform.rotation.x = q[0]
    t.transform.rotation.y = q[1]
    t.transform.rotation.z = q[2]
    t.transform.rotation.w = q[3]
    return t
def read_tf(t):
    pose=np.asarray((
        t.transform.translation.x,
        t.transform.translation.y,
        t.transform.translation.z
        ))
    quat=np.asarray((
        t.transform.rotation.x,
        t.transform.rotation.y,
        t.transform.rotation.z,
        t.transform.rotation.w
        ))
    
    return pose, quat


In [ ]:
res=segmentation_server.call()

In [ ]:
res

In [ ]:
poses=np.asarray(res.poses.data)

poses=poses.reshape((int(len(poses)/3) ,3     )      )

In [ ]:
head.set_named_target('neutral')

In [ ]:
#t=write_tf(res.poses.data,(0,0,0,1),'Target',"head_rgbd_sensor_depth_frame",0)
t=write_tf(poses[0,:],(0,0,0,1),'Target_tmp',"head_rgbd_sensor_depth_frame",0) ## CHOOSE A TARGET 

b_tf.sendTransform(t)
rospy.sleep(0.2)
tt=tfBuffer.lookup_transform('map','Target_tmp',rospy.Time(0))
rospy.sleep(0.2)
tt
pose,quat= read_tf(tt)
t=write_tf(pose,(0,0,0,1),'Target')
b_st.sendTransform(t)


In [ ]:
head = moveit_commander.MoveGroupCommander('head')
#gripper =  moveit_commander.MoveGroupCommander('gripper')
#arm=moveit_commander.MoveGroupCommander('arm')

In [ ]:
#whole_body=moveit_commander.MoveGroupCommander('whole_body')

In [ ]:
clear_octo_client()
arm_grasp_from_above = [0.3, -2.0, 0.0, -0.9939144210462025, -0.17365421548386273, 0.0]
wb_v=whole_body.get_current_joint_values()
wb_v[3:]=arm_grasp_from_above
succ = whole_body.go(wb_v)

In [ ]:
head.set_named_target('neutral')
head.go()
clear_octo_client()


In [ ]:
#
t=tfBuffer.lookup_transform('map','Target',rospy.Time(0))
pose , quat =  read_tf(t)
pose

In [ ]:
wb_gp=whole_body.get_current_pose()
wb_gp.pose.position.x= pose[0]
wb_gp.pose.position.y= pose[1]#
wb_gp.pose.position.z= pose[2]+0.3   ##Pre Grasp from above


In [ ]:
whole_body.set_pose_target(wb_gp)
plan=whole_body.plan()
if plan[0]:
    succ=whole_body.go()
else:
    print('No plan')

In [ ]:
succ

In [ ]:
t=tfBuffer.lookup_transform('hand_palm_link', 'Target',rospy.Time())

In [ ]:
pose,_=read_tf(t)
pose

In [ ]:
goal_p=whole_body.get_current_pose()
goal_p.pose.position.x=0.1
goal_p.pose.position.y=pose[1]
goal_p.pose.position.z=pose[2]+0.2
#goal_p.pose.orientation.w=quat[0]
#goal_p.pose.orientation.x=quat[1]
#goal_p.pose.orientation.y=quat[2]
#goal_p.pose.orientation.z=quat[3]



In [ ]:
whole_body.set_pose_target(goal_p)

In [ ]:
whole_body.plan()

In [ ]:
goal_p=whole_body.get_current_pose()

In [ ]:
whole_body.set_pose_target(goal_p)
whole_body.plan()

In [ ]:
whole_body.go()

In [ ]:
res.poses.data

In [ ]:
t=tfBuffer.lookup_transform('map','Target',rospy.Time())

In [ ]:
pose,quat=read_tf(t)

In [ ]:
goal_pose= Pose()

In [ ]:
goal_pose.position.x=pose[0]
goal_pose.position.y=pose[1]
goal_pose.position.z=pose[2]

In [ ]:
pose,goal_pose

In [ ]:
whole_body.get_current_pose()

In [ ]:
wb=whole_body.get_current_joint_values()

In [ ]:
wb[4]=0.1

In [ ]:
whole_body.go(wb)

In [ ]:
pose=whole_body.get_current_pose()

In [ ]:
type(pose.pose.orientation)

In [ ]:
tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))

In [ ]:
tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))

In [ ]:
gp=whole_body.get_current_pose()

In [ ]:

gp.pose.orientation
pose=gp
eu=tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))
#eu[1]=0
quat=tf.transformations.quaternion_from_euler(eu[0],eu[1],eu[2])

In [ ]:
eu=tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))

In [ ]:
tfBuffer.lookup_transform('hand_palm_link','New:static_Target',rospy.Time(0))

In [ ]:
#eu[1]=0
quat=tf.transformations.quaternion_from_euler(eu[0],eu[1],eu[2])
rot_quat=tf.transformations.quaternion_from_euler(0,0.5*np.pi,0.0)

In [ ]:
wb_gp=whole_body.get_current_pose()

In [ ]:
eu=tf.transformations.euler_from_quaternion((wb_gp.pose.orientation.w,wb_gp.pose.orientation.x,wb_gp.pose.orientation.y,wb_gp.pose.orientation.z))
quat=tf.transformations.quaternion_from_euler(eu[0],eu[1],eu[2])
rot_quat=tf.transformations.quaternion_from_euler(0,0.5*np.pi,0.0)
rotated=tf.transformations.quaternion_multiply(quat,rot_quat)



In [ ]:
wb_gp.pose.position.x-=0.1
wb_gp.pose.position.y+=0.1
wb_gp.pose.position.z-=0.1


In [ ]:
wb_gp.pose.orientation.w=rotated[0]
wb_gp.pose.orientation.x=rotated[1]
wb_gp.pose.orientation.y=rotated[2]
wb_gp.pose.orientation.z=rotated[3]

In [ ]:
whole_body.set_pose_target(wb_gp)

In [ ]:

plan=whole_body.plan()
plan

In [ ]:
if plan[0]:
    whole_body.go()

In [ ]:
whole_body.go()

In [ ]:
arm_grasp_from_above = [0.19263830140116414, -2.2668981568652917, -0.007358947463759424, -0.9939144210462025, -0.17365421548386273, 0.0]
wb_v=whole_body.get_current_joint_values()
wb_v[3:]=arm_grasp_from_above
succ = whole_body.go(wb_v)

In [ ]:

t=tfBuffer.lookup_transform('New:static_Target','hand_palm_link',rospy.Time(0))
pose , quat =  read_tf(t)
pose,quat
np.save('above_quat.npy',quat)

In [ ]:
t=tfBuffer.lookup_transform('map','Target',rospy.Time(0))
pose , quat =  read_tf(t)
pose,quat

In [ ]:
wb_gp=whole_body.get_current_pose()
wb_gp.pose.position.x= pose[0]
wb_gp.pose.position.y= pose[1]
wb_gp.pose.position.z= pose[2]+0.2


In [ ]:
whole_body.set_pose_target(wb_gp)
plan=whole_body.plan()
plan

In [ ]:

whole_body.go()

In [ ]:
wb_v=whole_body.get_current_joint_values()


In [ ]:
gripper = GRIPPER()

In [ ]:
gripper.close()

In [ ]:
gripper =  moveit_commander.MoveGroupCommander('gripper')

In [ ]:

|
tf_man.pub_static_tf()

In [ ]:
head.set_joint_values([0.0, -0.77])


In [ ]:
print ('rdy')

In [ ]:
brazo.set_named_target('go')
head.set_joint_values([0.0, -0.77])
res=segmentation_server.call()
if len(res.poses.data)==0: print( 'failed')
else:

    poses=np.asarray(res.poses.data)
    poses=poses.reshape((int(len(poses)/3) ,3     )      )  





    tf_man.pub_static_tf(pos=poses[0,:], point_name='target', ref='head_rgbd_sensor_rgb_frame')## which object to choose   #TODO
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(point_name='target', new_frame='map')
    rospy.sleep(0.3)
    pose, _ =tf_man.getTF('target')
    tf_man.pub_static_tf(pos=pose, point_name='Target')## which object to choose   #TODO


    head.set_named_target('neutral')

In [ ]:
poses=np.asarray(res.poses.data)
poses=poses.reshape((int(len(poses)/3) ,3     )      )  





tf_man.pub_static_tf(pos=poses[0,:], point_name='target', ref='head_rgbd_sensor_rgb_frame')## which object to choose   #TODO

In [ ]:
tf_man.change_ref_frame_tf(point_name='target', new_frame='map')


In [ ]:
pose, _ =tf_man.getTF('target')
tf_man.pub_static_tf(pos=pose, point_name='Target')## which object to choose   #TODO


In [ ]:
head.set_named_target('neutral')

In [ ]:
 clear_octo_client()
#############################10 * 2 *np.pi / 360 ####pi /2
arm_grasp_from_above = [0.2, -1.57, -0.13 , -1.57, 0.0, 0.0]
wb_v=whole_body.get_current_joint_values()
wb_v[3:]=arm_grasp_from_above
succ = whole_body.go(wb_v)   

In [ ]:
clear_octo_client()
pose, quat=tf_man.getTF('Target')

wb_gp=whole_body.get_current_pose()
wb_gp.pose.position.x= pose[0]
wb_gp.pose.position.y= pose[1]
wb_gp.pose.position.z= pose[2]+0.2

whole_body.set_pose_target(wb_gp)
plan=whole_body.plan()

In [5]:
pos,rot=tf_man.getTF(target_frame=,ref)
pos

False

[ WARN] [1683850128.534578537, 939.081000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 939.108000 according to authority unknown_publisher
[ WARN] [1683850128.536299704, 939.081000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 939.108000 according to authority /pose_integrator
[ WARN] [1683850128.536378567, 939.081000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 939.108000 according to authority /pose_integrator
[ WARN] [1683850128.536443377, 939.081000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 939.108000 according to authority /pose_integrator
[ WARN] [1683850128.536483425, 939.081000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 939.108000 according to authority /pose_integrator
[ WARN] [1683850128.536527903, 939.081000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

In [1]:
from smach_utils2 import *

[ WARN] [1683850054.652177057]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1683850054.667738475]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1683850054.667817523]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1683850054.670480183]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1683850054.670841728]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1683850054.853611500]: Kinematics solver doesn't support #attempts any

In [1]:
from smach_utils2 import *


[ WARN] [1683852101.287450327]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1683852101.292088975]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1683852101.292150409]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1683852101.293313113]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1683852101.293372339]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1683852101.447496782, 507.423000000]: Kinematics solver doesn't suppor

In [4]:
wb_gp=whole_body.get_current_pose()            
wb_gp.header.frame_id="Target"
wb_gp.pose.position.x=0.02669
wb_gp.pose.position.y=0.045
wb_gp.pose.position.z= 0.033
wb_gp.pose.orientation.w=-0.990
wb_gp.pose.orientation.x= 0.0246
wb_gp.pose.orientation.y= -0.0085
wb_gp.pose.orientation.z= 0.1345



[ WARN] [1683852401.137177061, 638.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 638.781000 according to authority unknown_publisher
[ WARN] [1683852401.137350706, 638.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 638.781000 according to authority /pose_integrator
[ WARN] [1683852401.137414483, 638.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 638.781000 according to authority /pose_integrator
[ WARN] [1683852401.137465173, 638.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 638.781000 according to authority /pose_integrator
[ WARN] [1683852401.137512084, 638.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 638.781000 according to authority /pose_integrator
[ WARN] [1683852401.137559074, 638.754000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1683852433.287461808, 653.102000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 653.102000 according to authority unknown_publisher
[ WARN] [1683852433.288070675, 653.102000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 653.102000 according to authority /pose_integrator
[ WARN] [1683852433.288143482, 653.102000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 653.102000 according to authority /pose_integrator
[ WARN] [1683852433.288198191, 653.102000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 653.102000 according to authority /pose_integrator
[ WARN] [1683852433.288243626, 653.102000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 653.102000 according to authority /pose_integrator
[ WARN] [1683852433.288291965, 653.102000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

shutdown request: [/smach] Reason: new node registered with same name


[ WARN] [1683852530.406576247, 695.391000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 695.418000 according to authority unknown_publisher
[ WARN] [1683852549.441358861, 703.600000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 703.611000 according to authority unknown_publisher
[ WARN] [1683852557.563189885, 707.052000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 707.075000 according to authority unknown_publisher
[ WARN] [1683852564.450110182, 709.905000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 709.924000 according to authority unknown_publisher
[ WARN] [1683852584.040703744, 718.474000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 718.493000 according to authority unknown_publisher
[ WARN] [1683852612.766870626, 731.012000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for fra

[ WARN] [1683853136.758195823, 960.679000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 960.692000 according to authority unknown_publisher
[ WARN] [1683853136.899912148, 960.731000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 960.754000 according to authority unknown_publisher
[ WARN] [1683853139.165071449, 961.755000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 961.790000 according to authority unknown_publisher
[ WARN] [1683853150.080064656, 966.512000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 966.528000 according to authority unknown_publisher
[ WARN] [1683853165.866234876, 973.422000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 973.442000 according to authority unknown_publisher
[ WARN] [1683853181.939284829, 980.567000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for fra

[ WARN] [1683853694.352007749, 1204.629000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1204.646000 according to authority unknown_publisher
[ WARN] [1683853698.708937533, 1206.609000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1206.638000 according to authority unknown_publisher
[ WARN] [1683853702.606785411, 1208.363000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1208.387000 according to authority unknown_publisher
[ WARN] [1683853710.027192531, 1211.912000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1211.942000 according to authority unknown_publisher
[ WARN] [1683853715.242547427, 1214.247000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1214.269000 according to authority unknown_publisher
[ WARN] [1683853727.298437086, 1219.701000000]: TF_REPEATED_DATA ignoring data with redundant timest

[ WARN] [1683854002.751272784, 1341.737000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1341.769000 according to authority unknown_publisher
[ WARN] [1683854036.591483630, 1356.566000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1356.610000 according to authority unknown_publisher
[ WARN] [1683854046.817584154, 1361.023000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1361.046000 according to authority unknown_publisher
[ WARN] [1683854065.883490433, 1369.247000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1369.274000 according to authority unknown_publisher
[ WARN] [1683854078.212273515, 1374.845000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1374.845000 according to authority unknown_publisher
[ WARN] [1683854085.530502034, 1378.127000000]: TF_REPEATED_DATA ignoring data with redundant timest

[ WARN] [1683854298.712170100, 1472.062000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1472.091000 according to authority unknown_publisher
[ WARN] [1683854299.131543851, 1472.247000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1472.285000 according to authority unknown_publisher
[ WARN] [1683854301.598558034, 1473.392000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1473.420000 according to authority unknown_publisher
[ WARN] [1683854304.665364684, 1474.777000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1474.812000 according to authority unknown_publisher
[ WARN] [1683854305.016552672, 1474.963000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1474.989000 according to authority unknown_publisher
[ WARN] [1683854316.009454140, 1479.801000000]: TF_REPEATED_DATA ignoring data with redundant timest

header: 
  seq: 0
  stamp: 
    secs: 515
    nsecs: 466000000
  frame_id: "odom"
pose: 
  position: 
    x: 0.02669
    y: 0.045
    z: 0.033
  orientation: 
    x: 0.0246
    y: -0.0085
    z: 0.1345
    w: -0.99

[ WARN] [1683852131.849285067, 520.534000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 520.560000 according to authority unknown_publisher
[ WARN] [1683852131.850083880, 520.534000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 520.560000 according to authority /pose_integrator
[ WARN] [1683852131.850155210, 520.534000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 520.560000 according to authority /pose_integrator
[ WARN] [1683852131.850196151, 520.534000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 520.560000 according to authority /pose_integrator
[ WARN] [1683852131.850238444, 520.534000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 520.560000 according to authority /pose_integrator
[ WARN] [1683852131.850287815, 520.534000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

In [5]:
aaa

In [1]:
from smach_ros import ActionServerWrapper

In [ ]:
from action_pickup_floor.msg import PickUpAction